In [ ]:
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
model = YOLO("runs/detect/train24/weights/best.pt").to("cuda")

In [ ]:
results = model.predict(r"C:\Users\Marcin\Videos\ganzgut.mp4")

In [ ]:
cap = cv2.VideoCapture(r"C:\Users\Marcin\Videos\ganzgut.mp4")
frame_number = 0
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
ret, frame = cap.read()
# if ret:
#     # Wyświetlanie klatki
#     cv2.imshow(f"Klatka nr {frame_number}", frame)
#     cv2.waitKey(0)  # Oczekiwanie na dowolny klawisz
#     cv2.destroyAllWindows()


In [ ]:
cv2.imwrite("results/frame.jpg", frame)

In [ ]:
players: list[np.ndarray] = []
for bbox in results[0].boxes.xyxy.tolist():
    player = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
    players.append(player)

In [ ]:
kits = []
for player in players:
    kit = player[player.shape[0]//6:player.shape[0]//2, int(player.shape[1]*0.35):int(player.shape[1]*0.65)]
    kits.append(kit)

In [ ]:
mean_colors: list[np.ndarray] = []
for kit in kits:
    mean_color = np.mean(kit, axis=(0, 1))
    color_image = np.zeros((100, 100, 3), dtype=np.uint8)
    color_image[:] = [int(mean_color[0]), int(mean_color[1]), int(mean_color[2])]
    mean_colors.append(color_image)

In [ ]:
mean_colors

In [ ]:
total_width_color = sum(color.shape[1] for color in mean_colors)
max_height_color = max(color.shape[0] for color in mean_colors)
output_image_colors = np.zeros((max_height_color, total_width_color, 3), dtype=np.uint8)

current_x_color = 0
for color in mean_colors:
    h, w = color.shape[:2]
    output_image_colors[0:h, current_x_color:current_x_color+w] = color
    current_x_color += w
    
cv2.imshow("Colors Side by Side", output_image_colors)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("colors.jpg", output_image_colors)

In [ ]:
total_width = sum(kit.shape[1] for kit in kits)
max_height = max(kit.shape[0] for kit in kits)

output_image = np.zeros((max_height, total_width, 3), dtype=np.uint8)

current_x = 0
for kit in kits:
    h, w = kit.shape[:2]
    output_image[0:h, current_x:current_x+w] = kit 
    current_x += w

cv2.imshow("kits Side by Side", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("kits.jpg", output_image)